# DBATools Log Shipping Demo
## Configure Log Shipping

In [ ]:
Remove-DbaDbLogShipping -PrimarySqlInstance 'localhost' -SecondarySqlInstance 'localhost\tony' -Database LSTest;
Remove-DbaDatabase -SqlInstance 'localhost\tony' -Database LSTestDR -Confirm:$false;

In [ ]:
Remove-Item -Path "C:\LogShipping\LSTest\*";
Remove-Item -Path "\\localhost\LogShippingDest\LSTest\*";
Remove-Item -Path "C:\LogShippingDest\LSTest\*";

$params = @{
 SourceSqlInstance = 'localhost'
 DestinationSqlInstance = 'localhost\tony'
 Database = 'LSTest'
 SharedPath= '\\localhost\LogShipping'
 LocalPath= 'C:\logshipping'
 BackupScheduleFrequencyType = 'daily'
 BackupScheduleFrequencyInterval = 1
 BackupScheduleDisabled = $false
 CompressBackup = $true
 CopyScheduleFrequencyType = 'daily'
 CopyScheduleFrequencyInterval = 1
 CopyScheduleDisabled = $false
 GenerateFullBackup = $true
 RestoreScheduleFrequencyType = 'daily'
 RestoreScheduleFrequencyInterval = 1
 RestoreScheduleDisable = $false
 SecondaryDatabaseSuffix = 'DR'
 CopyDestinationFolder = '\\localhost\LogShippingDest'
 Force = $true
 Standby = $true
 DisconnectUsers = $true
 }

Invoke-DbaDbLogShipping @params;


## Select from Primary and Secondary Tables

In [ ]:
$select = @"
SELECT 
@@SERVERNAME, * 
FROM TestTable
"@
$primary = "localhost";
$secondary = "localhost\tony"
$primarydb = "LSTest";
$secondarydb = "LSTestDR";

Invoke-Sqlcmd -ServerInstance $primary -Database $primarydb -Query $select;
Invoke-Sqlcmd -ServerInstance $secondary -Database $secondarydb -Query $select;


## Insert Into Primary Table

In [ ]:
$insert = @"
INSERT INTO TestTable
(Col2
,Col3)
VALUES
('Is'
,'Resilient')
"@
$primary = "localhost";
$primarydb = "LSTest";

Invoke-Sqlcmd -ServerInstance $primary -Database $primarydb -Query $insert;

## Select from Primary and Secondary Tables

In [ ]:
$select = @"
SELECT 
@@SERVERNAME, * 
FROM TestTable
"@
$primary = "localhost";
$secondary = "localhost\tony"
$primarydb = "LSTest";
$secondarydb = "LSTestDR";

Invoke-Sqlcmd -ServerInstance $primary -Database $primarydb -Query $select;
Invoke-Sqlcmd -ServerInstance $secondary -Database $secondarydb -Query $select;


## Execute Log Shipping Jobs

In [ ]:
Start-DbaAgentJob -SqlInstance "localhost" -Job LSBackup_LSTest -Wait;
Start-DbaAgentJob -SqlInstance "localhost\tony" -Job LSCopy_localhost_LSTest -Wait;
Start-DbaAgentJob -SqlInstance "localhost\tony" -Job LSRestore_localhost_LSTest -Wait;

## Select from Primary and Secondary Tables

In [ ]:
$select = @"
SELECT 
@@SERVERNAME, * 
FROM TestTable
"@
$primary = "localhost";
$secondary = "localhost\tony"
$primarydb = "LSTest";
$secondarydb = "LSTestDR";

Invoke-Sqlcmd -ServerInstance $primary -Database $primarydb -Query $select;
Invoke-Sqlcmd -ServerInstance $secondary -Database $secondarydb -Query $select;


## Stop Primary SQL Server Service and Recover Secondary Database

This step simulates a failure of the primary SQL instance and recovers the database on the secondary.

- Stops the SQL Server service on the localhost
- Kills any open connections to the LSTestDR database on localhost\\tony
- Runs Invoke-DbaDbLogShipRecovery to bring the secondary database online

In [ ]:
Stop-Service -Name MSSQLSERVER -Force;
Stop-DbaProcess -SqlInstance 'localhost\tony' -Database LSTestDR;

Invoke-DbaDbLogShipRecovery -SqlInstance 'localhost\tony' -Force;

## Select from New Primary Table

In [ ]:
$select = @"
SELECT 
@@SERVERNAME, * 
FROM TestTable
"@

$primary = "localhost\tony"
$primarydb = "LSTestDR";

Invoke-Sqlcmd -ServerInstance $primary -Database $primarydb -Query $select;

## Insert Into New Primary Table

In [ ]:
$insert = @"
INSERT INTO TestTable
(Col2
,Col3)
VALUES
('Is'
,'Failed Over')
"@
$primary = "localhost\tony";
$primarydb = "LSTestDR";

Invoke-Sqlcmd -ServerInstance $primary -Database $primarydb -Query $insert;

## Select from New Primary Table After Insert

In [ ]:
$select = @"
SELECT 
@@SERVERNAME, * 
FROM TestTable
"@

$primary = "localhost\tony"
$primarydb = "LSTestDR";

Invoke-Sqlcmd -ServerInstance $primary -Database $primarydb -Query $select;

# Restart Primary SQL Service

In [ ]:
Start-Service -Name MSSQLSERVER;
Start-Service -Name SQLSERVERAGENT;